# Question 4: General Theory/Application

_No need to be verbose, it's not fun for anyone_

1. What part of S**L**A**M** did this project deal with? Why? What does the other part deal with and how would it generally work, given that you only have LIDAR scans, RGB video stream, and noisy pose data for a moving robot?


2. Loop closures play an important role in reducing drift, how would you go about detecting these?


3. Explain the structure of your Jacobian. Is the pose-graph fully connected? Why/Why not?


4. With what you know now, how would you describe and differentiate the SLAM frontend and backend? Why do we need to optimise our poses/map in the first place - where does the noise come from/why?

## Answer 1: Parts of SLAM

The project deals with the **Localization** aspect of SLAM, using an optimization problem (_pose graph optimization_). Here, the vertices (states) were given, and they had to be optimized using odometry and loop closure constraints.

The other part is **Mapping** and it deals with tracking the pose of landmarks in a map (usually a global coordinate system). This entails adding state vectors of each landmark and successively updating their estimated position using a filter (like Bayes filter or a Kalman Filter). Usually, these new states (landmarks) have no motion model (they're assumed to be fixed in the global frame). SLAM entails simultaneous optimization (or finding) of both the position(s) of the robot in global frame (**localization**) and the pose of landmarks in the global frame (**mapping**).


Given a robot with noisy motion data and sensors, one might run some feature detection or object detection algorithm to find landmarks (a pose estimation algorithm may also be needed to find the poses / positions of landmarks in the local environment). One approach could be to keep a track of a certain number of features that have very less noise in detection. These could also be cross-checked with some loop-closure algorithms (if there is a loop closure, you would also expect similar landmarks to be around). 

Note that local detections can be transformed to the global frame using homogeneous transformations and the current estimate of states. As the state estimates get better, so does the map.

When using a filter, a model of the measurements given the map and pose may be needed (along with the motion model). Beliefs could be updated using a method similar to Bayes' Rule.

## Answer 2: Loop Closure

Odometry can deviate and accumulate errors over time. Loop closures allow us to reduce localization error upon detection. There could be methods based on

- Clustering features and running neighborhood based pattern matching: where a graph of features and neighborhood information can be stored (in a quickly searchable manner), and new features sensed can be queried to find earlier occurrences.
- Backtracking state-space trees can also be used for detecting loop closures. A method employing branch-and-bound approach is explored in [Wolfgang Hess, et al. 2016](https://doi.org/10.1109/ICRA.2016.7487258)
- Local shape and color information can be used, while extending the bag-of-words method. A such visual bag of words method is demonstrated in [A. Angeli, et al. 2008](https://doi.org/10.1109/TRO.2008.2004514). Another reference can be [Nishant Kejriwal, et al.](https://doi.org/10.1016/J.ROBOT.2015.12.003)


## Answer 3: Jacobian and Pose Graph connectivity

The Jacobian was visualized as an image in `Question 2` (2D SLAM).

The Jacobian as an image is shown below

![Jacobian Matrix as an image](./results/q4/jac_1.jpg)

It is observable that it is very sparse (many values are 0), indicating that there are very few states involved in each constraint (two in our case). This is because the jacobian is simply the derivative of the constraint equations w.r.t. the states.

The jacobian matrix is a 420 by 360 matrix (there are 420 constraints and 360 states). Out of the 420 constraints, 357 are odometry (= 119 * 3), 60 are loop closure (= 20 * 3), 3 are zero constraint (= 3 * 1). 